# <center>Тема 6
## <center> Прогнозирование задержек вылетов

Ваша задача – побить бенчмарки в [соревновании](https://www.kaggle.com/c/flight-delays-2017) на Kaggle Inclass. Подробных инструкций не будет, будут только рекомендации. Конечно, с помощью Xgboost. Надеюсь, на данном этапе курса вам достаточно бросить полтора взгляда на данные, чтоб понять, что это тот тип задачи, в которой затащит Xgboost.

In [24]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score

from scipy import sparse
from sklearn.model_selection import GridSearchCV
import xgboost as xgb
from sklearn import metrics

In [2]:
train = pd.read_csv('data/flight_delays_train.csv')
test = pd.read_csv('data/flight_delays_test.csv')

In [3]:
train.head()

,Month,DayofMonth,DayOfWeek,DepTime,UniqueCarrier,Origin,Dest,Distance,dep_delayed_15min
0,c-8,c-21,c-7,1934,AA,ATL,DFW,732,N
1,c-4,c-20,c-3,1548,US,PIT,MCO,834,N
2,c-9,c-2,c-5,1422,XE,RDU,CLE,416,N
3,c-11,c-25,c-6,1015,OO,DEN,MEM,872,N
4,c-10,c-7,c-6,1828,WN,MDW,OMA,423,Y


In [4]:
test.head()

,Month,DayofMonth,DayOfWeek,DepTime,UniqueCarrier,Origin,Dest,Distance
0,c-7,c-25,c-3,615,YV,MRY,PHX,598
1,c-4,c-17,c-2,739,WN,LAS,HOU,1235
2,c-12,c-2,c-7,651,MQ,GSP,ORD,577
3,c-3,c-25,c-7,1614,WN,BWI,MHT,377
4,c-6,c-6,c-3,1505,UA,ORD,STL,258


Итак, надо по времени вылета самолета, коду авиакомпании-перевозчика, месту вылета и прилета и расстоянию между аэропортами вылета и прилета предсказать задержку вылета более 15 минут. В качестве простейшего бенчмарка возьмем логистическую регрессию и два признака, которые проще всего взять: `DepTime` и `Distance`. У такой модели результат – 0.68202 на LB. 

In [5]:
X_train, y_train = train[['Distance', 'DepTime']].values, train['dep_delayed_15min'].map({'Y': 1, 'N': 0}).values
X_test = test[['Distance', 'DepTime']].values

X_train_part, X_valid, y_train_part, y_valid = train_test_split(X_train, 
                                                                y_train, 
                                                                test_size=0.3, 
                                                                random_state=17)

In [6]:
logit = LogisticRegression(random_state=17)

logit.fit(X_train_part, y_train_part)
logit_valid_pred = logit.predict_proba(X_valid)[:, 1]

roc_auc_score(y_valid, logit_valid_pred)

0.6795697123357751

In [7]:
logit.fit(X_train, y_train)
logit_test_pred = logit.predict_proba(X_test)[:, 1]

pd.Series(logit_test_pred, 
          name='dep_delayed_15min').to_csv('logit_2feat.csv', 
                                           index_label='id', 
                                           header=True)

Рекомендации:
- признаки `Distance` и  `DepTime` можно использовать  без изменений
- создать признак "маршрут" из исходных `Origin` и `Dest`
- к признакам `Month`, `DayofMonth`, `DayOfWeek`, `UniqueCarrier` и "маршрут" применить OHE-преобразование (`LabelBinarizer`)
- выделить отложенную выборку
- обучать логистическую регрессию и градиентный бустинг (xgboost), гиперпараметры бустинга настраивать на кросс-валидации, сначала те, что отвечают за сложность модели, затем число деревьев фиксировалось равным 500 и настраивался шаг градиентного спуска
- с помощью `cross_val_predict` делать прогнозы обеих моделей на кросс-валидации (именно предсказанные вероятности), настраивать линейную смесь ответов логистической регрессии и градиентного бустинга вида $w_1 * p_{logit} + (1 - w_1) * p_{xgb}$, где $p_{logit}$ – предсказанные логистической регрессией вероятности класса 1, $p_{xgb}$ – аналогично. Вес $w_1$ подбирать вручную. 
- в качестве ответа для тестовой выборки брать аналогичную комбинацию ответов двух моделей, но уже обученных на всей обучающей выборке.


Удачи!

In [8]:
# Преобразование результата
y_train = train['dep_delayed_15min'].apply(lambda x: 0 if x == 'N' else 1)

# Объединение отбытия и назначения в путь
train['path'] = train['Origin'] + train['Dest']
test['path'] = test['Origin'] + test['Dest']

# Удаление значений выше
train.drop(['Origin', 'Dest', 'dep_delayed_15min'], axis=1, inplace=True)
test.drop(['Origin', 'Dest'], axis=1, inplace=True)

# Замена категориальных данных на числовые
cat_cols = ['Month', 'DayofMonth', 'DayOfWeek', 'UniqueCarrier', 'path']
dummy_train = pd.get_dummies(train[cat_cols])
dummy_new = pd.get_dummies(test[cat_cols])

# Создание нового объекта в соответствии с новой последовательностью меток
dummy_new = dummy_new.reindex(columns = dummy_train.columns, fill_value=0)

# Объединяем данные
train = pd.concat([train[['DepTime', 'Distance']], dummy_train], axis=1)
test = pd.concat([test[['DepTime', 'Distance']], dummy_new], axis=1)

# Преобразуем в часы
train['DepHour'] = train['DepTime'] // 100
test['DepHour'] = test['DepTime'] // 100

# Удаление значений выше
train.drop('DepTime', axis=1, inplace=True)
test.drop('DepTime', axis=1, inplace=True)

# Ненулевые значения разряженной матрицы
train_sp = sparse.csr_matrix(train)
test_sp = sparse.csr_matrix(test)

In [9]:
train.head()

,Distance,Month_c-1,Month_c-10,Month_c-11,Month_c-12,Month_c-2,Month_c-3,Month_c-4,Month_c-5,Month_c-6,...,path_XNALAX,path_XNALGA,path_XNAORD,path_XNASLC,path_YAKCDV,path_YAKJNU,path_YUMIPL,path_YUMLAX,path_YUMPHX,DepHour
0,732,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,19
1,834,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,15
2,416,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,14
3,872,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,10
4,423,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,18


In [12]:
xgbc = XGBClassifier(max_depth=6, n_estimators=500)
xgbc.fit(train_sp, y_train)

[13:34:44] WARNING: D:\Build\xgboost\xgboost-1.4.2.git\src\learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=500, n_jobs=8, num_parallel_tree=1, random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [21]:
answer = xgbc.predict_proba(test_sp)
sub = pd.read_csv('logit_2feat.csv')
sub['dep_delayed_15min'] = answer[:, 1]
sub.to_csv('subm1.csv', index=None)

## -

In [25]:
X_tr, X_val, y_tr, y_val = train_test_split(train_sp, y_train, test_size=0.33)

In [26]:
def modelfit(alg, dtrain, target, useTrainCV=True, cv_folds=5, early_stopping_rounds=50):
    
    if useTrainCV:
        xgb_param = alg.get_xgb_params()
        xgtrain = xgb.DMatrix(dtrain, label=target)
        cvresult = xgb.cv(xgb_param, xgtrain, num_boost_round=alg.get_params()['n_estimators'], nfold=cv_folds,
            metrics='auc', early_stopping_rounds=early_stopping_rounds, verbose_eval=False)
        alg.set_params(n_estimators=cvresult.shape[0])
    
    #Fit the algorithm on the data
    alg.fit(dtrain, target, eval_metric='auc')
        
    #Predict training set:
    dtrain_predictions = alg.predict(dtrain)
    dtrain_predprob = alg.predict_proba(dtrain)[:,1]
        
    #Print model report:
    print ("\nModel Report")
    print ("Accuracy : %.4g" % metrics.accuracy_score(target, dtrain_predictions))
    print ("AUC Score (Train): %f" % metrics.roc_auc_score(target, dtrain_predprob))

In [27]:
xgb1 = XGBClassifier(
    learning_rate =0.1,
    n_estimators=1000,
    max_depth=5,
    min_child_weight=1,
    gamma=0,
    subsample=0.8,
    colsample_bytree=0.8,
    objective= 'binary:logistic',
    nthread=4,
    scale_pos_weight=1,
    seed=27)
modelfit(xgb1, train_sp, y_train)


Model Report
Accuracy : 0.8257
AUC Score (Train): 0.781408


In [28]:
pred = xgb1.predict_proba(test_sp)[:, 1]

In [29]:
sub = pd.read_csv('logit_2feat.csv')
sub['dep_delayed_15min'] = pred
sub.to_csv('subm2.csv', index=None)

In [31]:
param_test = {
    'min_child_weight': [6,8,10],
    'n_estimators': [1000, 1200],
    'max_depth': [5, 6, 7]
}
gsearch = GridSearchCV(estimator = XGBClassifier(learning_rate=0.1, n_estimators=140, max_depth=4,
 min_child_weight=2, gamma=0, subsample=0.8, colsample_bytree=0.8,
 objective= 'binary:logistic', nthread=4, scale_pos_weight=1), 
 param_grid=param_test, scoring='roc_auc', cv=5, verbose=1)
gsearch.fit(train_sp, y_train)

Fitting 5 folds for each of 18 candidates, totalling 90 fits
[14:18:20] WARNING: D:\Build\xgboost\xgboost-1.4.2.git\src\learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


c:\users\metar\appdata\local\programs\python\python39\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
c:\users\metar\appdata\local\programs\python\python39\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[14:18:45] WARNING: D:\Build\xgboost\xgboost-1.4.2.git\src\learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


c:\users\metar\appdata\local\programs\python\python39\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[14:19:14] WARNING: D:\Build\xgboost\xgboost-1.4.2.git\src\learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


...

c:\users\metar\appdata\local\programs\python\python39\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[15:04:44] WARNING: D:\Build\xgboost\xgboost-1.4.2.git\src\learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


c:\users\metar\appdata\local\programs\python\python39\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[15:05:21] WARNING: D:\Build\xgboost\xgboost-1.4.2.git\src\learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


GridSearchCV(cv=5,
             estimator=XGBClassifier(base_score=None, booster=None,
                                     colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=0.8, gamma=0, gpu_id=None,
                                     importance_type='gain',
                                     interaction_constraints=None,
                                     learning_rate=0.1, max_delta_step=None,
                                     max_depth=4, min_child_weight=2,
                                     missing=nan, monotone_constraints=None,
                                     n_estimators=140, n_jobs=None, nthread=4,
                                     num_parallel_tree=None, random_state=None,
                                     reg_alpha=None, reg_lambda=None,
                                     scale_pos_weight=1, subsample=0.8,
                                     tree_method=None, val

In [32]:
gsearch.best_params_

{'max_depth': 5, 'min_child_weight': 10, 'n_estimators': 1000}

In [33]:
modelfit(gsearch.best_estimator_, train_sp, y_train)


Model Report
Accuracy : 0.8265
AUC Score (Train): 0.784786


In [34]:
pred = gsearch.best_estimator_.predict_proba(test_sp)[:, 1]
sub = pd.read_csv('logit_2feat.csv')
sub['dep_delayed_15min'] = pred
sub.to_csv('subm3.csv', index=None)

{'max_depth': 5, 'min_child_weight': 10, 'n_estimators': 1000}